# Space missions data understanding, visualization and analysis

Who does not love Space? 
This DataSet was scraped from https://nextspaceflight.com/launches/past/?page=1 and includes all the space missions since the beginning of Space Race (1957) 

![Image](https://i1.wp.com/matmatch.com/blog/wp-content/uploads/2019/03/AdobeStock_80273384-compressor.jpeg?resize=2000%2C1125&ssl=1)

## Introduction 

Ever since the start of world war when the USSR launched the first ever manmade sattelite into the space called the Sputnik, the world has taken great interests in trying to explore beyond the planet. Rocket science, Cosmology, Astronomy are the epitome of engineering and science that require extreme levels of theoretical as well as experimental work.

A lot of mathematics goes behind when and where should a space launch take place for reaching it's destination at least possible resistance and with highest probability of success. At the same time, extreme levels of engineering is done to simulate the similar space conditions back on earth and test the launch vehicles for any possible failures. All these space missions require years of hard work, research and tests for success.

## Importing the libraries

In [ ]:
import numpy as np  # Numpy helps us to do complex and high-level mathematical functions on large,
                    # multi-dimentional array and matrices
    
import pandas as pd # Pandas offers us data structures and operations for manipulating numerical 
                    #tables and time series
    
import matplotlib.pyplot as plt # Matplotlib helps us to embed plots into applications, commonly used
                                # for data visualisation
    
import seaborn as sns # Seaborn provides a high-level interface for drawing attractive and informative 
                      # statistical graphics.
    
sns.set_theme(style='darkgrid') # Setting the style of seaborn plots to 'darkgrid'

import plotly.express as px # Plotly is yet another data visualisation and interactive open-source library.
                            # Plotly graph objects are a high-level interface to plotly which are easy to use

## Reading and making a very basic understanding of the dataset

In [ ]:
df = pd.read_csv("../input/all-space-missions-from-1957/Space_Corrected.csv")# Reads the .csv file and converts 
                                                                             # to pandas Dataframe Object

In [ ]:
df.head()# Shows top rows of the dataset (default value-5)

In [ ]:
df.info() # Gathering more knowledge about the data like number of rows and columns, datatype and not-null values

In [ ]:
df.shape # shape of the dataframe object

Let us quickly try to understand each of the columns and what they mean.

- **Company name** : The space organisation undertaking the mission
- **Location** : The point of spacecraft launch on earth
- **Datum** : Date and time of liftoff
- **Detail** : Name and type of the spaceship
- **Status of rocket** : Whether the space craft is still under commission and active in it's mission
- **Rocket** : Cost of the mission in million dollars
- **Status Mission** : Whether the mission was successful.

Any other columns are not required and we will drop them.

## Feature Extraction and Data Cleaning

In [ ]:
df.head()

In [ ]:
df=df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)# Dropping the unwanted rows

In [ ]:
df.columns # For displaying all the columns

If we notice the Rocket column carefully, we will see that there is a space before the word 'Rocket'. We will fix this before proceeding

In [ ]:
df=df.rename(columns={' Rocket':'Rocket'})# Renaming the column

In [ ]:
df.columns

If we look at the location column, we will find that name of the launch site is combined with the country. We must seperate out the country to have a more meaningfull insight into the data.

In [ ]:
df["Country"] = df["Location"].apply(lambda location: location.split(", ")[-1])
# We cant directly use a function on a dataframe object. So, we use apply method.

In [ ]:
df.head()

As we did for the location column, we also need to seperate the year from the Datum column. We can always extract the year by using split but we have a very simple workaround by using a method in pandas library.
to_datatime converts time in any format to a proper datetime format from which details such as year,month and hour can be extracted very easily.

In [ ]:
df['DateTime'] = pd.to_datetime(df['Datum']) 
df['Year'] = df['DateTime'].apply(lambda datetime: datetime.year)# Extracting the year from Datum column and 
                                                                 # putting it in DateTime Column

In [ ]:
df.head()

In [ ]:
df["Launch_Site"] = df["Location"].apply(lambda location: ", ".join(location.split(", ")[:-1]))
# Seperating the Launch Site information and putting it in Launch_Site Column

In [ ]:
df.head()

In [ ]:
df['Rocket'].unique()# For all unique values for the column Rocket

Now we see that there are commas for some numbers, so we have to take out that comma so that we can convert it to float

In [ ]:
df['Rocket'] = df['Rocket'].fillna(0.0).str.replace(',', '')

In [ ]:
df['Rocket'].unique()

## Data Visualisation

### Company

#### Countplot of Space Missions by Companies

In [ ]:
plt.figure(figsize=(10,18),dpi=150)
ax = sns.countplot(y="Company Name", data=df, order=df["Company Name"].value_counts().index)
ax.set_xscale("log")# Sets the scale to logarithmic value
plt.title('All Companies',size=25)
plt.show()

**RVSN USSR is the company to launch the most rockets. Also, RSVN USSR does not exist anymore after the disintegration of USSR. Roscosmos is the new space agency of Russia which sits on the 16th place in our list.**

#### Top 20 space companies

In [ ]:
df['Count']=1
df_comps=df.groupby('Company Name')['Count'].sum().reset_index().sort_values(by='Count',ascending=False)
df_comps=df_comps.head(20)
sns.catplot('Company Name','Count',data=df_comps,palette='Set2',kind='bar',aspect=2,height=8)
plt.title('Top 20 space companies',size=25)
plt.xticks(rotation=60)
plt.xlabel('Company name')
plt.yticks(size=15)
plt.show()

#### The top 15 companies in satellite mission

In [ ]:
labels=list(df["Company Name"].value_counts()[:15].keys())
sizes=df["Company Name"].value_counts()[:15]
explode=[]
for i in labels:
    explode.append(0.075)
plt.figure(figsize=(6,6),dpi=150)
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']
plt.pie(sizes,labels=labels,explode=explode, autopct='%1.1f%%', startangle=90, shadow=False,
        textprops={'fontsize': 6},colors=colors)
plt.title("Top 15 Companies in Satellite Mission", fontsize=10, loc="right")
plt.axis("equal")
plt.tight_layout()

### Location

#### Countplot of Missions by Countries

In [ ]:
plt.figure(figsize=(10,10),dpi=200)
ax = sns.countplot(y="Country", data=df, order=df["Country"].value_counts().index)
ax.set_xscale("log")
plt.title('All Countries',size=25)
plt.show()

**We infer that Russia had the most space missions, also quite evident from the fact that RSVN USSR is the company to have the most space launches. USA was just short of them while India sits at 7th place.**

#### Top 10 Countries leading the space launches

In [ ]:
df_countries=df.groupby('Country')['Count'].sum().reset_index().sort_values(by='Count',ascending=False)
df_countries=df_countries.head(10)
plt.figure(figsize=(10,8),dpi=150)
sns.catplot('Country','Count',data=df_countries,aspect=2,height=8,kind='bar',palette='Set2')
plt.title('Top 10 nations leading the space launches',size=15)
plt.xticks(size=15,rotation=60)
plt.xlabel('Country',size=15)
plt.ylabel('Count',size=15)
plt.yticks(size=15)
plt.show()

#### Countplot of Missions by Launch Sites

In [ ]:
plt.figure(figsize=(10,40),dpi=200)
ax = sns.countplot(y="Launch_Site", data=df, order=df["Launch_Site"].value_counts().index)
ax.set_xscale("log")
plt.title('All Launch Sites',size=25)
plt.show()

**From this plot, we can see that Site 31/6, Baikonur Cosmodrone is the most used launch site and it is one of the testing sites of RSVN USSR.**

### Datum

#### Countplot of launches by year

In [ ]:
plt.figure(figsize=(10,18),dpi=150)
sns.countplot(data=df,y='Year')
plt.title("Launches By Year")
plt.show()

**So we infer from this plot that 1972 was the year in which most rockets were launched. It was the year when Apollo 16 was launched and the space race was at its peak. 2018 was the year with most launches after 1972.**

#### Frequency of Launches by Year

In [ ]:
plt.figure(figsize=(10,6),dpi=150)
sns.distplot(df['Year'],kde=True,rug=True,bins=80)
plt.title("Frequency of Launches By Year")
plt.show()

#### Number of Launches By Month

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_title('No. of Launches by Month', fontsize=20)
month_df=pd.DataFrame()
df['Month']=df['Datum'].apply(lambda datum: datum.split()[1])
month_df['Month']=df['Month'].value_counts().index
month_df['Launches']=df['Month'].value_counts().values
order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
sorter = [order.index(i) for i in month_df['Month']]
month_df['Sorter']=sorter
sns.lineplot(x = 'Month', y = "Launches", data=month_df.sort_values(by='Sorter'), sort=False)
ax.set_xlabel('Month', fontsize=15)
ax.set_ylabel('No. of Launches')
plt.show()

**We can see that moat missions happen between November and December. This can be due to the fact that weather is quite good at this time of the year**

#### Launches in 2020 by different Companies

In [ ]:
df_2020=df[df['Year']==2020]
sns.catplot('Company Name',data=df_2020,kind='count',aspect=2,height=8,palette='Set2')
plt.yticks(np.arange(20))
plt.title('2020 launches',size=25)
plt.xlabel('Company name',size=20)
plt.xticks(size=15,rotation=45)
plt.yticks(size=15)
plt.ylabel('Number of missions',size=15)
plt.show()

**Launches in 2020 are led by Elon Musk's Space-X, while we see that new companies like Rocket Lab, Virgin Orbit are also launching quite a few **

### Money

#### Money Spent by Companies

In [ ]:
df.Rocket = df.Rocket.apply(lambda x: str(x).strip()).astype('float64')
data = df.groupby(['Company Name'])['Rocket'].sum().reset_index()
data = data[data['Rocket'] > 0].sort_values(by='Rocket', ascending=False)
data.columns = ['Company', 'Money']
plt.figure(figsize=(10,10),dpi=150)
plt.title("Money Spent By Companies",size=15)
ax=sns.barplot(data=data,x='Company',y='Money')
ax.set_yscale("log")
plt.xticks(rotation=60)
plt.show()

**While RSVN USSR may have the most launches but NASA tops the charts when it comes to spending Money.**

### Mission and Rocket Status

#### Countplot of Mission Status

In [ ]:
plt.figure(figsize=(3,3),dpi=150)
ax = sns.countplot(data=df,y='Status Mission',order=df["Status Mission"].value_counts().index, palette="Set2")
ax.set_xscale("log")
plt.title("Count of Mission Status")
plt.show()

**From the plot, its clearly visible that most of the missions were successful and very few were a prelaunch failure**

#### Pie Chart of Mission Status

In [ ]:
fig = px.pie(df,names='Status Mission', title='Status of Mission',width=600, height=400)
fig.show()

**Roughly 90% of the missions till date were a success.**

#### Countplot of Missions by Rocket Retirement Status

In [ ]:
plt.figure(figsize=(3,3),dpi=150)
sns.countplot(x="Status Rocket",data=df,order=df["Status Rocket"].value_counts().index,palette='Set2')
plt.title("Count of Rocket Status")
plt.show()

**Clearly, most of the launch vehicles used or the satellites launched are retired.**

# ISRO Data Analysis
![](https://www.isro.gov.in/sites/default/files/galleries/गैलरी/19take.jpg)

In [ ]:
df_isro=df[df["Company Name"]=="ISRO"]

In [ ]:
df_isro.info()

In [ ]:
df_isro.head(10)

#### Status Mission for ISRO

In [ ]:
plt.figure(figsize=(3,3),dpi=150)
sns.countplot(data=df_isro,y='Status Mission',order=df["Status Mission"].value_counts().index, palette="Set2")
plt.title("ISRO Mission Status",size=15)
plt.show()

**There are no prelaunch failures and most missions were successful.**

#### COuntplot of launches by Year

In [ ]:
plt.figure(figsize=(10,18),dpi=150)
sns.countplot(data=df_isro,y='Year')
plt.title("ISRO Launches By Year")
plt.show()

**Now lets see which is the most used launch pad for isro**

In [ ]:
plt.figure(figsize=(3,3),dpi=150)
ax = sns.countplot(y="Launch_Site", data=df_isro, order=df_isro["Launch_Site"].value_counts().index)
plt.title('All ISRO Launch Sites',size=15)
plt.yticks(rotation=60)
plt.rc('xtick',labelsize=4)
plt.rc('ytick',labelsize=4)
plt.show()

**So, the most used site is First Launch pad, Satish Dhawan Space Center.**

In [ ]:
df.to_csv(r'Space_Missions_Cleaned.csv',index=False)# Saving the cleaned df object into a csv file.